The user inputs a name for the metadata folder, the state FIPS code, and the list of county FIPS codes from which data is required.

In [1]:
import pandas as pd
import geopandas as gpd
import replicaEVSE.geography as geo
%reload_ext autoreload
%autoreload 1

datadir = '../../data/'

Pull block group shape data for a sate via its fips number

In [ ]:
gdf = geo.get_bg_shapefiles('53')

make a mapping between blockgroup and county

read in the made bg file

In [2]:
gdf = pd.read_pickle(datadir+'/blockgroup_boundaries.pkl')

In [3]:
gdf

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,53,029,971400,2,530299714002,Block Group 2,G5030,S,2814636,1797525,+48.2062765,-122.5296535,"POLYGON ((-122.54657 48.20801, -122.54250 48.2..."
1,53,029,971400,3,530299714003,Block Group 3,G5030,S,22645103,62368,+48.2125091,-122.4991207,"POLYGON ((-122.52732 48.18245, -122.52727 48.1..."
2,53,029,971500,1,530299715001,Block Group 1,G5030,S,14190551,3407670,+48.2523579,-122.4561538,"POLYGON ((-122.48503 48.25507, -122.48216 48.2..."
3,53,029,971500,2,530299715002,Block Group 2,G5030,S,13421381,9435655,+48.2145613,-122.4526433,"POLYGON ((-122.48178 48.16779, -122.48169 48.1..."
4,53,029,971600,1,530299716001,Block Group 1,G5030,S,13176547,2594118,+48.1716032,-122.5137524,"POLYGON ((-122.53917 48.19050, -122.54435 48.1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4778,53,015,001100,2,530150011002,Block Group 2,G5030,S,311974,41091,+46.1488880,-122.9092078,"POLYGON ((-122.91383 46.14731, -122.91382 46.1..."
4779,53,015,001100,3,530150011003,Block Group 3,G5030,S,651187,0,+46.1398107,-122.9027207,"POLYGON ((-122.91179 46.13775, -122.91152 46.1..."
4780,53,015,001100,4,530150011004,Block Group 4,G5030,S,220828,0,+46.1381419,-122.9127996,"POLYGON ((-122.91549 46.13344, -122.91514 46.1..."
4781,53,015,001100,5,530150011005,Block Group 5,G5030,S,301290,0,+46.1360225,-122.9080286,"POLYGON ((-122.91351 46.13303, -122.91324 46.1..."


get the mapping between county names and fips codes

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_counties_in_Washington'

tables = pd.read_html(url)

# Select the desired table
table_index = 1  # Adjust this index based on the table you want to scrape
wiki_df = tables[table_index]

In [5]:
wiki_df.columns

Index(['County', 'FIPS code', 'County seat[11]', 'Est.[11][12]',
       'Formed from[12][13]', 'Etymology', 'Population (2022)[14]',
       'Land area[11]', 'Map'],
      dtype='object')

In [6]:
wiki_df['COUNTYFP'] = wiki_df['FIPS code'].astype(str).str.zfill(3)
wiki_df[['COUNTYFP', 'County']]

,COUNTYFP,County
0,001,Adams County
1,003,Asotin County
2,005,Benton County
3,007,Chelan County
4,009,Clallam County
5,011,Clark County
6,013,Columbia County
7,015,Cowlitz County
8,017,Douglas County
9,019,Ferry County


In [8]:
cnty_df = pd.merge(left=gdf, right=wiki_df[['COUNTYFP', 'County']], on='COUNTYFP', how='left')

In [10]:
cnty_df['BLOCKGROUP'] = cnty_df['GEOID']
cnty_df['destination_bgrp'] = cnty_df['GEOID']

In [12]:
cnty_df.to_csv(datadir+'blockgroup_counties.csv', index=False)

In [13]:
cnty_df.

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,County,BLOCKGROUP,destination_bgrp
0,53,029,971400,2,530299714002,Block Group 2,G5030,S,2814636,1797525,+48.2062765,-122.5296535,"POLYGON ((-122.54657 48.20801, -122.54250 48.2...",Island County,530299714002,530299714002
1,53,029,971400,3,530299714003,Block Group 3,G5030,S,22645103,62368,+48.2125091,-122.4991207,"POLYGON ((-122.52732 48.18245, -122.52727 48.1...",Island County,530299714003,530299714003
2,53,029,971500,1,530299715001,Block Group 1,G5030,S,14190551,3407670,+48.2523579,-122.4561538,"POLYGON ((-122.48503 48.25507, -122.48216 48.2...",Island County,530299715001,530299715001
3,53,029,971500,2,530299715002,Block Group 2,G5030,S,13421381,9435655,+48.2145613,-122.4526433,"POLYGON ((-122.48178 48.16779, -122.48169 48.1...",Island County,530299715002,530299715002
4,53,029,971600,1,530299716001,Block Group 1,G5030,S,13176547,2594118,+48.1716032,-122.5137524,"POLYGON ((-122.53917 48.19050, -122.54435 48.1...",Island County,530299716001,530299716001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5338,53,015,001100,2,530150011002,Block Group 2,G5030,S,311974,41091,+46.1488880,-122.9092078,"POLYGON ((-122.91383 46.14731, -122.91382 46.1...",Cowlitz County,530150011002,530150011002
5339,53,015,001100,3,530150011003,Block Group 3,G5030,S,651187,0,+46.1398107,-122.9027207,"POLYGON ((-122.91179 46.13775, -122.91152 46.1...",Cowlitz County,530150011003,530150011003
5340,53,015,001100,4,530150011004,Block Group 4,G5030,S,220828,0,+46.1381419,-122.9127996,"POLYGON ((-122.91549 46.13344, -122.91514 46.1...",Cowlitz County,530150011004,530150011004
5341,53,015,001100,5,530150011005,Block Group 5,G5030,S,301290,0,+46.1360225,-122.9080286,"POLYGON ((-122.91351 46.13303, -122.91324 46.1...",Cowlitz County,530150011005,530150011005
